In [1]:
import numpy as np
import pandas as pd
import requests
import re
import json
from os import walk
from multiprocessing.pool import ThreadPool

In [2]:
URL = "https://s3.amazonaws.com/irs-form-990/index_2019.json"

response = requests.get(URL)
with open("index_2019.json", 'wb') as f:
    f.write(response.content)

In [3]:
with open("index_2019.json") as f:
    data = json.load(f)
    data = data[list(data.keys())[0]]

In [4]:
df = pd.DataFrame.from_dict(data)
df.tail(5)

,EIN,TaxPeriod,DLN,FormType,URL,OrganizationName,SubmittedOn,ObjectId,LastUpdated
396212,550761361,201906,93493296013149,990,https://s3.amazonaws.com/irs-form-990/20194296...,GLEN DALE VOLUNTEER FIRE DEPARTMENT,2019-12-06,201942969349301314,2019-12-12T23:31:55.5172618Z
396213,526078441,201906,93493296013399,990,https://s3.amazonaws.com/irs-form-990/20194296...,COMMON CAUSE,2019-12-06,201942969349301339,2019-12-12T23:31:55.5172618Z
396214,460377419,201906,93493296013449,990,https://s3.amazonaws.com/irs-form-990/20194296...,NEW DAWN ENTERPRISES,2019-12-05,201942969349301344,2019-12-12T23:31:55.5172618Z
396215,205239366,201812,93493296015049,990,https://s3.amazonaws.com/irs-form-990/20194296...,JOHN PAUL II MEDICAL RESEARCH INSTITUTE,2019-12-05,201942969349301504,2019-12-12T23:31:55.5172618Z
396216,810481055,201906,93493303009239,990,https://s3.amazonaws.com/irs-form-990/20193303...,ROI FOUNDATION INC,2019-12-06,201933039349300923,2019-12-12T23:31:55.5172618Z


In [5]:
df.shape

(396217, 9)

In [6]:
def download_url(url):
    file_name_start = url.rfind('/') + 1
    file_name = url[file_name_start:]
    output_dir = "/home/meso/git_repo/exploring_990/data_2019/"
    file_name = output_dir + file_name
    
    r = requests.get(url, stream=True)
    if r.status_code == requests.codes.ok:
        with open(file_name, 'wb') as f:
            f.write(r.content)
    return url

urls = []
for i in range(df.shape[0]): 
    urls.append(data[i]['URL'])
    
results = ThreadPool(100).imap_unordered(download_url, urls)

In [23]:
f = []
for (dirpath, dirnames, filenames) in walk('/home/meso/git_repo/exploring_990/data_2019/'):
    f.extend(filenames)
    break

In [24]:
u = list(df['URL'])
len(u)

396217

In [25]:
file_names_list = []

for i in range(len(u)):
    file_loc = u[i].rfind("/") + 1
    file_name = u[i][file_loc:]
    file_names_list.append(file_name)

In [26]:
print(len(file_names_list), str(":"), len(f))

396217 : 396187


In [27]:
diff = list(set(file_names_list) - set(f))
len(diff)

0

In [28]:
file_loc = u[i].rfind("/") + 1
url_pre = u[0][:file_loc]

In [29]:
urls = []
for i in range(len(diff)):
    url = url_pre + diff[i]
    urls.append(url)

In [30]:
for url in urls:
    download_url(url)

In [31]:
f = []
for (dirpath, dirnames, filenames) in walk('/home/meso/git_repo/exploring_990/data_2019/'):
    f.extend(filenames)
    break

In [32]:
len(f)

396187

In [33]:
diff = list(set(file_names_list) - set(f))
len(diff)

0

In [34]:
def list_duplicates(seq):
    seen = set()
    seen_add = seen.add
    return [idx for idx, item in enumerate(seq) if item in seen or seen_add(item)]

In [35]:
print(list_duplicates(file_names_list))

[85406, 85408, 85410, 85412, 85414, 85998, 86689, 86690, 86691, 86692, 86693, 86694, 86695, 86696, 86697, 86698, 86700, 86701, 86702, 86703, 86704, 86705, 86706, 86707, 86708, 86709, 86710, 89282, 89283, 91562]


In [36]:
dup_df = df.iloc[list_duplicates(file_names_list)]

In [37]:
dup_ein = pd.DataFrame(dup_df['EIN'])

In [38]:
dup_ein

,EIN
85406,411742282
85408,060870134
85410,931069099
85412,912046528
85414,953447952
85998,362924251
86689,330592140
86690,951688546
86691,270987719
86692,136108649


In [39]:
duplicates = pd.DataFrame([])

for dup in range(dup_ein.shape[0]):
    ein = dup_ein.iloc[dup]['EIN']
    duplicates = duplicates.append(df[df['EIN'] == ein])

In [40]:
duplicates

,EIN,TaxPeriod,DLN,FormType,URL,OrganizationName,SubmittedOn,ObjectId,LastUpdated
39535,411742282,201712,93493318122468,990,https://s3.amazonaws.com/irs-form-990/20181318...,CHILDRENS MENTAL HEALTH SERVICES REACH INC,2019-02-06,201813189349312246,2019-04-15T18:19:53
85406,411742282,201712,93493318122468,990,https://s3.amazonaws.com/irs-form-990/20181318...,CHILDRENS MENTAL HEALTH SERVICES REACH INC,2019-03-26,201813189349312246,2019-04-15T18:19:53
39530,060870134,201806,93493318122618,990,https://s3.amazonaws.com/irs-form-990/20181318...,NEW HAVEN JEWISH COMMUNITY COUNCIL HOUSING COR...,2019-02-06,201813189349312261,2019-04-15T18:19:53
85408,060870134,201806,93493318122618,990,https://s3.amazonaws.com/irs-form-990/20181318...,NEW HAVEN JEWISH COMMUNITY COUNCIL HOUSING COR...,2019-03-26,201813189349312261,2019-04-15T18:19:53
39543,931069099,201712,93493318122768,990,https://s3.amazonaws.com/irs-form-990/20181318...,BROADWAY ROSE THEATRE CO,2019-02-06,201813189349312276,2019-04-15T18:19:53
...,...,...,...,...,...,...,...,...,...
89282,274541012,201712,93493318122818,990,https://s3.amazonaws.com/irs-form-990/20181318...,SOCIAL GOOD LABS INCORPORATED,2019-03-26,201813189349312281,2019-04-15T18:19:53
85409,061014706,201806,93493318122668,990,https://s3.amazonaws.com/irs-form-990/20181318...,RIDGEFIELD WORKSHOP FOR THE PEFORMING ARTS INC,2019-02-06,201813189349312266,2019-04-15T18:19:53
89283,061014706,201806,93493318122668,990,https://s3.amazonaws.com/irs-form-990/20181318...,RIDGEFIELD WORKSHOP FOR THE PEFORMING ARTS INC,2019-03-26,201813189349312266,2019-04-15T18:19:53
86699,136090722,201712,93493318122568,990,https://s3.amazonaws.com/irs-form-990/20181318...,NEW YORK PATHOLOGICAL SOCIETY,2019-03-26,201813189349312256,2019-04-15T18:19:53


In [43]:
duplicates['URL'].iloc[0]

'https://s3.amazonaws.com/irs-form-990/201813189349312246_public.xml'

In [44]:
duplicates['URL'].iloc[1]

'https://s3.amazonaws.com/irs-form-990/201813189349312246_public.xml'